## [TGS Salt Identification Challenge](https://www.kaggle.com/c/tgs-salt-identification-challenge)

## Changelog <a name='A'></a>
<div align="right">[>>TOC](#TOC)</div>

# csum
2 .  import imgaug
https://arxiv.org/pdf/1706.05587.pdf

nadam . = 0.38!
adam, 30pochs score = 0.38! - adam werkt niet goed?
DEPTH 32x32 = 0.756 (40 epochs)
DEPTH 32x32 = 0.753 (10 epochs) . 
DEPTH 64x64 = 0.736

#
batch=32, rmsprop, patience=13, lrdecay=4, val_loss  
RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0),
/// {'adam': Adam(lr=LR, beta_1=0.9, beta_2=0.999, clipnorm=8.),
test score=0.763

#
batch=48, rmsprop, patience=13, lrdecay=4, val_loss  
'rmsprop': RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0),
test score=0.762

#
batch=16, rmsprop, patience=13, lrdecay=4, val_loss  
test score=0.753

#
batch=24, rmsprop, patience=11, lrdecay=4, val_loss  
test score=0.764

#
batch =32
metric = val_miou
test score=0.745

#
UP_CONV = False
RES = True  
'adadelta'

21/aug/2018  
 - initial commit  

## Table of contents: <a name='TOC'></a>
[Changelog](#A)  
[Sources](#B)  
[About](#C)  
[U-Net model](#D)  
[Setup notebook](#E)  
[Data Exploration](#F)  
[Visualize images and masks (overlayed)](#G)  
[Train validation split](#H)  
[Custom metrics](#I)  
[Model architecture](#J)  
[Build model](#K)  
[Train model](#L)  
[Learning curves](#M)  
[Check performance on validation set](#N)  
[Scoring](#O)  
[Post-processing: blur and threshold](#Q)   
[Predict test set](#P)   
[Submission](#R)   

## Sources that earn credits <a name='B'></a>
<div align="right">[>>TOC](#TOC)</div>

I got initial inspiration and insights for this notebook from:  

[Alexander Liao](https://www.kaggle.com/alexanderliao/u-net-bn-aug-strat-dice/notebook)  -   Model architecture , dice loss  
[Jesper Dramsch](https://www.kaggle.com/jesperdramsch/intro-to-seismic-salt-and-how-to-geophysics)   - Seismic intro  

## About
(*Jesper Dramsch*)<a name='C'></a>
<div align="right">[>>TOC](#TOC)</div>

Seismic data is a neat thing. You can imagine it like an ultra-sound of the subsurface. However, in an ultra-sound, we use much smaller wavelengths to image our body. Seismic data usually has wavelengths around 1m to 100m. That has some physical implications, but for now, we don't have to deal with that. It's just something to keep in mind while thinking about resolution. 

Imaging salt has been a huge topic in the seismic industry, basically since they imaged salt the first time. The Society of Exploration geophysicist alone has over 10,000 publications with the [keyword salt](https://library.seg.org/action/doSearch?AllField=salt). Salt bodies are important for the hydrocarbon industry, as they usually form nice oil traps. So there's a clear motivation to delineate salt bodies in the subsurface. If you would like to do a deep dive, you can see [this publication](https://www.iongeo.com/content/documents/Resource%20Center/Articles/INT_Imaging_Salt_tutorial_141101.pdf)

Seismic data interpreters are used to interpreting on 2D or 3D images that have been heavily processed. The standard work of [seismic data analysis](https://wiki.seg.org/wiki/Seismic_Data_Analysis) is open access.
You'll find sections on Salt in there as well (https://wiki.seg.org/wiki/Salt-flank_reflections and https://wiki.seg.org/wiki/Salt_flanks). The seismic itself is pretty "old" in the publication, and you're dealing with data that is less noisy here, which is nice.

[![Seismic Data with salt CC-BY-SA Yilmaz](https://wiki.seg.org/images/1/14/Ch05_fig0-1.png)](https://wiki.seg.org/wiki/Salt-flank_reflections#/media/File:Ch05_fig0-1.png)
Caption: Figure 5.0-1  Conflicting dips associated with salt flanks: (a) CMP stack without dip-moveout correction; (b) time migration of the stack in (a); (c) the stack with dip-moveout correction; (d) time migration of the stack in (c). CC-BY-SA Yilmaz.

Interpretation on seismic images has long used texture attributes, to identify better and highlight areas of interest. These can be seen like feature maps on the texture of the seismic. For salt, you will notice that the texture in the salt masks is rather chaotic, where the surrounding seismic is more "striped". You can think of Earth as layered. Sand gets deposited on top of existing sand. In comes salt, which is behaving very much, unlike other rocks. There is an entire research branch dedicated to salt tectonics, that is the movement of salt in the subsurface. To give you the gist, these salt diapirs form from salt layers somewhere else that were under much pressure. These started to flow (behave ductile) and find a way into other layers above. I have written a bit about salt on [my blog](http://the-geophysicist.com/the-showroom-data-for-my-thesis).

One common seismic attribute is called "chaos" or "seismic disorder". So if you talk to cynic geophysicists, you'll hear "that deep learning better outperform the Chaos attribute". A good starting point is [this publication](http://www.chopraseismic.com/wp-content/uploads/2016/08/Chopra_Marfurt_TLE_Aug2016-LowRes.pdf).

Recently, geoscience has started to adopt deep learning, and it has seen a clear boom, particularly in imaging salt. Code for automatic seismic interpretation can be found here: 

+ https://github.com/waldeland/CNN-for-ASI
+ https://github.com/bolgebrygg/MalenoV
+ https://github.com/crild/facies_net

You will notice that these solutions load a specific SEG-Y file, which luckily we don't have to bother with. TGS provided some nice PNG files instead. However, you can glean some information from them how to approach seismic data. If you find you need some geophysical helpers, you can [import Bruges](https://github.com/agile-geoscience/bruges)


## U-Net model<a name='D'></a>
<div align="right">[>>TOC](#TOC)</div>

The seismic images resemble X-rays and Ultrasound scans. The U-Net model is a CNN used for Biomedical Image Segmentation.   

Please read the paper: <a href="https://arxiv.org/pdf/1505.04597.pdf"> U-Net: Convolutional Networks for Biomedical Image Segmentation</a>) 

<a href="https://github.com/jocicmarko/ultrasound-nerve-segmentation">Another interesting read</a> from the Kaggle Ultrasound Nerve Segmentation competition.</p>

<p><img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" alt=""></p>



## Setup notebook<a name='E'></a>
<div align="right">[>>TOC](#TOC)</div>

### Notebook mode

In [ ]:
# Set Commit/Development mode
BATCH_SIZE = 24
EPOCHS = 5
COMMIT = False
DEV = not COMMIT
DBG = False

In [ ]:
import os
import gc
import sys
import glob
import random
import resource
from memory_profiler import profile

import pandas as pd
import numpy as np
from itertools import chain
import cv2

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
%matplotlib inline
import ipywidgets as ipy

from tqdm import tqdm_notebook

from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize

from sklearn.model_selection import train_test_split
import tensorflow as tf
from imgaug import augmenters as iaa

from keras.models import Model, load_model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Dropout, BatchNormalization, UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose, AtrousConvolution2D
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Nadam
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras import backend as K

### Profiling helpers

In [ ]:
def mem_used():
    """Memory used"""
    return round(resource.getrusage(resource.RUSAGE_SELF)[2] * 10/1028 / 10, 1)

def mem_fun(fun, **kwargs):
    """"""
    mem_start = mem_used()
    _ = fun(**kwargs)
    print(f'memory used by function: {(mem_used() - mem_start):.1f}mb')
    
# 
def test_alloc_size(n, m, c):
    """Check memory allocation needed for train/test set"""
    imgs_test_alloc = list(range(n))
    for i in range(n):
        imgs_test_alloc[i] = np.ones((m, TGT_SIZE, TGT_SIZE, c)) * random.randint(0,100)
    return imgs_test_alloc
    
if DBG:
    mem_fun(test_alloc_size, n=1, m=18000, c=2)
    print(f'notebook memory used: {mem_used()}mb')
    ## memory used by function: 4482.3mb for (1800, 128, 1288, 2)
    ## notebook memory used: 4805.7mb

nb_mem = mem_used()
f'Initial memory used by this notebook: {nb_mem}mb'

### Set file locations
For convenience when working from different locations.

In [ ]:
DATA_DIR = '.'
path = './input'
path_train = f'{path}/train'
path_test = f'{path}/test'
imgs_train = f'{path}/train/images'
masks_train = f'{path}/train/masks'
imgs_test = f'{path}/test/images'

In [ ]:
# Full path names
train_img_filepaths = sorted(glob.glob(f'{imgs_train}/*'))
train_mask_filepaths = sorted(glob.glob(f'{masks_train}/*'))
test_img_filepaths = sorted(glob.glob(f'{imgs_test}/*'))

In [ ]:
# Get first glimpse at images and masks
if DEV:
    @ipy.interact(idx=ipy.IntSlider(min=1, max=len(train_img_filepaths)), value=10, step=1)
    def plot(idx):
        """Navigate thru images and masks"""
        img = np.array(load_img(train_img_filepaths[idx], grayscale=True))
        mask = np.array(load_img(train_mask_filepaths[idx], grayscale=True))
        fig, axs = plt.subplots(1,2, figsize=(16,8))
        axs[0].imshow(img, cmap="Greys")
        axs[1].imshow(mask, cmap="Greens")
        plt.show()

In [ ]:
# from https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/61720
# To consider exclude from training, not knowing if such masks are in LB test set
BAD_MASKS =[
    '1eaf42beee','33887a0ae7','33dfce3a76','3975043a11','39cd06da7d','483b35d589','49336bb17b','4ef0559016',
    '4fbda008c7','4fdc882e4b','50d3073821','53e17edd83','5b217529e7','5f98029612','608567ed23','62aad7556c',
    '62d30854d7','6460ce2df7','6bc4c91c27','7845115d01','7deaf30c4a','80a458a2b6','81fa3d59b8','8367b54eac',
    '849881c690','876e6423e6','90720e8172','916aff36ae','919bc0e2ba','a266a2a9df','a6625b8937','a9ee40cf0d',
    'aeba5383e4','b63b23fdc9','baac3469ae','be7014887d','be90ab3e56','bfa7ee102e','bfbb9b9149','c387a012fc',
    'c98dfd50ba','caccd6708f','cb4f7abe67','d0bbe4fd97','d4d2ed6bd2','de7202d286','f0c401b64b','f19b7d20bb',
    'f641699848','f75842e215','00950d1627','0280deb8ae','06d21d76c4','09152018c4','09b9330300','0b45bde756',
    '130229ec15','15d76f1672','182bfc6862','23afbccfb5','24522ec665','285f4b2e82','2bc179b78c','2f746f8726',
    '3cb59a4fdc','403cb8f4b3','4f5df40ab2','50b3aef4c4','52667992f8','52ac7bb4c1','56f4bcc716','58de316918',
    '640ceb328a','71f7425387','7c0b76979f','7f0825a2f0','834861f1b6','87afd4b1ca','88a5c49514','9067effd34',
    '93a1541218','95f6e2b2d1','96216dae3b','96523f824a','99ee31b5bc','9a4b15919d','9b29ca561d','9eb4a10b98',
    'ad2fa649f7','b1be1fa682','b24d3673e1','b35b1b412b','b525824dfc','b7b83447c4','b8a9602e21','ba1287cb48',
    'be18a24c49','c27409a765','c2973c16f1','c83d9529bd','cef03959d8','d4d34af4f7','d9a52dc263','dd6a04d456',
    'ddcb457a07','e12cd094a6','e6e3e58c43','e73ed6e7f2','f6e87c1458','f7380099f6','fb3392fee0','fb47e8e74e','febd1d2a67']


## Visualize images and masks (overlayed)<a name='G'></a>
<div align="right">[>>TOC](#TOC)</div>

Helper functions:
- upsample image
- ploting overlayed images in grid - for using throughout notebook

In [ ]:
IMG_SIZE = 101   # original/raw image size
TGT_SIZE = 128   # tunable hyperparam: model/input image size {128, 256, 512} - for this 14gb RAM kernel 128 is max. size

In [ ]:
def upsample(img, dim=TGT_SIZE, interpolation=cv2.INTER_LINEAR):
    """Resize image to target shape(model_input) or back to original shape
    
    INTER_NEAREST - a nearest-neighbor interpolation
    INTER_LINEAR - a bilinear interpolation (used by default)
    INTER_AREA - resampling using pixel area relation. It may be a preferred method for image decimation, as it gives moire’-free results. But when the image is zoomed, it is similar to the INTER_NEAREST method.
    INTER_CUBIC - a bicubic interpolation over 4x4 pixel neighborhood
    INTER_LANCZOS4 - a Lanczos interpolation over 8x8 pixel neighborhood
    To shrink an image, it will generally look best with CV_INTER_AREA interpolation, whereas to enlarge an image, 
    it will generally look best with CV_INTER_CUBIC(slow) or CV_INTER_LINEAR(faster but still looks OK).
    """
    if img.shape[0] == dim:
        return img
    return cv2.resize(img, (dim, dim), interpolation=cv2.INTER_CUBIC)

In [ ]:
def plot_imgs_masks(imgs, masks, **kwargs):
    """Visualize seismic images with their salt area mask(green) and optionally salt area prediction(pink). 
    The prediction mask can be either in probability-mask or binary-mask form(based on threshold)
    Arguments:
    imgs: vector(-1, h, w, ch)
    masks: vector(-1, h, w, ch)
    """
    depth = kwargs.get('depth', None)
    coverage = kwargs.get('coverage', None)
    cov_class = kwargs.get('cov_class', None)
    preds_valid = kwargs.get('preds_valid', None)
    thres = kwargs.get('thres', None)
    grid_width = kwargs.get('grid_width', 10)
    zoom = kwargs.get('zoom', 1.5)
    title = kwargs.get('title', "Green: Salt area mask \nTop-left: coverage class, top-right: salt coverage, bottom-left: depth")
    
    grid_height = 1 + (len(imgs)-1) // grid_width
    fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*zoom, grid_height*zoom))
    axes = axs.ravel()
    
    for i, (img, mask) in enumerate(zip(imgs, masks)):
        
        ax = axes[i] #//grid_width, i%grid_width]
        _ = ax.imshow(img[..., 0], cmap="Greys")
        _ = ax.imshow(mask[..., 0], alpha=0.3, cmap="Greens")
        
        if preds_valid is not None:
            pred = preds_valid[i]
            pred = pred[..., 0]
            if thres is not None:
                pred = np.array(np.round(pred > thres), dtype=np.float32)
                iou = f'IoU: {IoU(mask, pred)}'
                _ = ax.imshow(pred, alpha=0.3, cmap="Oranges")
                _ = ax.text(2, img.shape[0]-2, iou, color="k")
            else:
                _ = ax.imshow(pred, alpha=0.3, cmap="Oranges")
            
        if depth is not None:
            _ = ax.text(2, img.shape[0]-2, f'depth: {depth[i].round(3)}', color="k")
        if (coverage is not None) and (cov_class is not None):   
            _ = ax.text(2, 2, f'{coverage[i].round(3)}({cov_class[i]})', color="k", ha="left", va="top")
        _ = ax.set_yticklabels([])
        _ = ax.set_xticklabels([])
        _ = plt.axis('off')
    plt.suptitle(title, y=1+.5/grid_height, fontsize=20)
    plt.tight_layout();

### Visualise sample - get a feel of the data

In [ ]:
if DEV:
    N = 100
    samples = random.sample(list(range(N)), N)
    X = [np.array(load_img(train_img_filepaths[sample], grayscale=True))[...,np.newaxis] for sample in samples]
    Y = [np.array(load_img(train_mask_filepaths[sample], grayscale=True))[...,np.newaxis] for sample in samples]
    plot_imgs_masks(X, Y) #, coverage=Y_coverages[sample], cov_class=Y_cov_class[sample], depth=X_norm_depth[sample])

## Data Exploration<a name='F'></a>
<div align="right">[>>TOC](#TOC)</div>

Load and build train and test set.

 - contemplating to use depth as a layer, this explains the stacking in `images_d`.
 - checking depth
 - checking coverage
 - checking depth - coverage relationship
 - visualize seismic images with masks

### Load csv files

These files contain image filenames(without .png) and seismic depths of the images.  
 - train.csv - filenames of train set  
 - depths.csv - depths of all images test & train set

In [ ]:
train_df_ = pd.read_csv(f'{path}/train.csv', index_col="id", usecols=[0])
depths_df_ = pd.read_csv(f'{path}/depths.csv', index_col="id") # train and test
train_df = train_df_.join(depths_df_)
test_df = depths_df_[~depths_df_.index.isin(train_df_.index.values)]

# Indices
train_indices = train_df.index.values
test_indices = test_df.index.values

In [ ]:
# Free up some RAM
del depths_df_
print(f'notebook memory used: {mem_used()}mb')

### Depth

 - checking the distribution
 
*As per below: depth is 'normal' distributed and train and test set have same distribution.*

In [ ]:
depth = train_df.z
mean_depth, std_depth, max_depth = depth.mean(), depth.std(), depth.max()
X_norm_depth = (depth - mean_depth) / std_depth

In [ ]:
if DEV:
    _ = sns.distplot(train_df.z, label="Train")
    _ = sns.distplot(test_df.z, label="Test")
    _ = plt.legend()
    _ = plt.title("Depth distribution")

### EDA dept, coverage, std, mean

 - checking the correlation between depth and coverage
 
 Stretching and offsetting the normalized salt coverage for visual comparison reason only
 
*As per below: no pattern or correlation visible, depth and coverage are unrelated. Depth might still be a factor in the prediction, e.g. the structure/grain might relate to depth.*

In [ ]:
# Coverage
Y_cov = [(np.sum(upsample(np.array(load_img(f'{masks_train}/{name}.png', grayscale=True)), TGT_SIZE) // 255) / TGT_SIZE**2,      # salt coverage
              (np.sum(upsample(np.array(load_img(f'{masks_train}/{name}.png', grayscale=True)), TGT_SIZE) // 255) / TGT_SIZE**2  # salt coverage class
                  - .01) * 100//10 + 1) for name in tqdm_notebook(train_indices)]


In [ ]:
Y_coverages, Y_cov_class = zip(*Y_cov)
Y_coverages = np.array(Y_coverages)
Y_cov_class = np.array(Y_cov_class)

In [ ]:
# Standard deviation in images
X_std = [np.std(np.array(load_img(f'{imgs_train}/{name}.png', grayscale=True)))  
         for name in tqdm_notebook(train_indices)]

In [ ]:
# Mean in images
X_mean = [np.mean(np.array(load_img(f'{imgs_train}/{name}.png', grayscale=True)))  
         for name in tqdm_notebook(train_indices)]

In [ ]:
np_stack = np.stack([train_indices, np.array(X_mean), np.array(X_std), Y_coverages, Y_cov_class]).T
np_stack.shape

In [ ]:
df = pd.DataFrame(np_stack, columns=['id', 'mean', 'stddev', 'coverage', 'cov_class']).set_index('id')
df.stddev = pd.to_numeric(df.stddev) #.astype(np.float)
df['mean'] = pd.to_numeric(df['mean'])
df.coverage = pd.to_numeric(df.coverage)
df['cov_norm'] = (df.coverage - np.mean(df.coverage)) / np.std(df.coverage)
df.cov_class = pd.to_numeric(df.cov_class).astype(np.int8)
df['depth'] = depth
df['depth_norm'] = X_norm_depth
df['std_class'] = np.digitize(df.stddev, [0., 5., 10., 20., 30., 40., 50., 60., 70, 1000.])
df.head()
df.info()

In [ ]:
_ = sns.lmplot('stddev', 'cov_norm', data=df)

In [ ]:
if DEV:
    # Pairplot
    _ = sns.pairplot(df.loc[:, ['depth', 'cov_norm']], size=6)
    _ = sns.pairplot(df.loc[df.cov_class!=0, ['depth', 'cov_norm']], size=6)

In [ ]:
if DEV:
    # Pairplot
    _ = sns.pairplot(df.loc[df.cov_class!=0, ['mean', 'cov_norm']], size=6)

In [ ]:
sns.jointplot('stddev', 'coverage', data=df, color='b', kind='reg');

In [ ]:
sns.jointplot('mean', 'coverage', data=df, color='b', kind='reg');

In [ ]:
if DEV:
    plt.figure(figsize=(20,8))
    with sns.axes_style(style=None):
        sns.violinplot('cov_class', 'stddev', data=df, inner="quartile") #
#                        split=True);

In [ ]:
if DEV:
    plt.figure(figsize=(20,8))
    with sns.axes_style(style=None):
        sns.violinplot('std_class', 'coverage', data=df, inner="quartile") #
#                        split=True);

In [ ]:
if DEV:
    plt.figure(figsize=(20,8))
    with sns.axes_style(style=None):
        sns.violinplot('std_class', 'mean', data=df, inner="quartile") #
#                        split=True);

In [ ]:
plt.figure(figsize=(20,30))
g = sns.PairGrid(df, vars=['depth_norm', 'cov_norm', 'mean', 'stddev'], hue='cov_class', palette='RdBu_r')
g.map(plt.scatter, alpha=0.5)
g.add_legend();

In [ ]:
if DEV:
    std_zero = set(df[df.stddev<5].index) | set(df[df.stddev>45].index) | set(df[df['mean']<50].index) | set(df[df['mean']>200].index)
    N = 200
    samples = random.sample(std_zero, min(len(std_zero), N))
    X = [np.array(load_img(f'{imgs_train}/{sample}.png', grayscale=True))[...,np.newaxis] for sample in samples]
    Y = [np.array(load_img(f'{masks_train}/{sample}.png', grayscale=True))[...,np.newaxis] for sample in samples]
    plot_imgs_masks(X, Y, title='Stand dev is zero') #, coverage=Y_coverages[sample], cov_class=Y_cov_class[sample], depth=X_norm_depth[sample])

### Coverage class

*As per below: there are 8-10 times more seismic images with 0-10% salt areas. Stratification in train-validation split must prevent overfitting.*

In [ ]:
if DEV:
    bincount = np.bincount(list(Y_cov_class))
    _ = plt.figure(figsize=(15, 6))
    _ = sns.distplot(Y_cov_class, label="Train", kde=False, bins=len(bincount))
    _ = plt.legend()
    _ = plt.title("Coverage distribution")
    print(bincount)

## Data augmentation

In [ ]:
def downsample(imgs, dim=64):
    """"""
    return [resize(img, (dim, dim), mode='constant', preserve_range=True) for img in imgs]

def img_func(images, random_state, parents, hooks):
    """Lambda cumsum transform"""
    for i, img in enumerate(images):
        images[i] = (np.float32(img) - img.mean()).cumsum(axis=0)
    return images

def keypoint_func(keypoints_on_images, random_state, parents, hooks):
    return keypoints_on_images

def augment_xy(X, Y, dim=128):
    """Augment X and Y same way"""
    # Augmenting
    # https://imgaug.readthedocs.io/en/latest/source/augmenters.html
    seq = iaa.Sequential([
#         iaa.Scale(dim), # scales to (dim, dim)
        iaa.Fliplr(0.5), # horizontally flip 50% of images
        iaa.OneOf([
            iaa.Sometimes(0.5, iaa.Noop()), # iaa.Lambda(img_func, keypoint_func)), returns negative values on masks
            iaa.SomeOf(2, [
            iaa.Noop(),
            iaa.Noop(),
            iaa.GaussianBlur(sigma=(0.0, 2.0)),
            iaa.Affine(rotate=(-10, 10), translate_percent={"x": (-0.25, 0.25)},  mode='symmetric',  cval=(0)),
            iaa.PiecewiseAffine(scale=(0.02, 0.06), mode='edge', cval=(0)),
        ]) 
                  ]),
        # More as you want ...
    ])
    
    # Convert the stochastic sequence of augmenters to a deterministic one.
    # The deterministic sequence will always apply the exactly same effects to the images.
    seq_det = seq.to_deterministic()
    
    X_aug = np.array(seq_det.augment_images(X))[...,np.newaxis]
#     print(X_aug[:, :64, :64, :].shape)
#     X_aug2 = np.array(downsample(seq_det.augment_images(X)))[...,np.newaxis]
    Y_aug = np.array(seq_det.augment_images(Y))[...,np.newaxis]
    X_dict = {'img': X_aug, 
              'csum': X_aug[:, :dim, :dim, :], # np.ones(len(X)*dim*dim).reshape(-1, dim, dim, 1) * 0.001,
              'feat': np.ones(len(X)).reshape(-1, 1) * 0.01 * np.random.normal(0, 0.1, 1)}
    return X_dict, Y_aug
        


## Data loader - generator

In [ ]:
def data_gen(batch=BATCH_SIZE, indices=train_indices, dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train):
    """Unlimited stochastic augmentation from training set"""
    while True:
        rand_indices = np.random.choice(indices, batch)  # unlimited sampling with replacement - stochastic
        X = [upsample(np.array(load_img(f'{imgs_train}/{name}.png', grayscale=True)), dim) / 255 for name in rand_indices]
        Y = [upsample(np.array(load_img(f'{masks_train}/{name}.png', grayscale=True)), dim) / 255 for name in rand_indices]
        yield augment_xy(X, Y, 64)
        
def val_gen(batch=BATCH_SIZE, indices=train_indices, dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train):
    """Unlimited stochastic sampling from validation set"""
    while True:
        rand_indices = np.random.choice(indices, batch)  # unlimited sampling with replacement - stochastic
        X = [upsample(np.array(load_img(f'{imgs_train}/{name}.png', grayscale=True)), dim) / 255 for name in rand_indices]
        Y = [upsample(np.array(load_img(f'{masks_train}/{name}.png', grayscale=True)), dim) / 255 for name in rand_indices]
        yield np.array(X)[...,np.newaxis], np.array(Y)[...,np.newaxis]

In [ ]:
# Visual sanity check
XY = next(data_gen(batch=40, indices=train_indices.tolist(), dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train))
plot_imgs_masks(XY[0]['img'], XY[1], title='Visual sanity check augmenting images and masks')

## Custom metrics and loss functions<a name='I'></a>
<div align="right">[>>TOC](#TOC)</div>

Custom metrics can be passed at the compilation step. The function would need to take (y_true, y_pred) as arguments and return a single tensor value. A metric function is similar to a loss function, except that the results from evaluating a metric are not used when training the model.

In [ ]:
def mean_iou(Y_true, Y_pred, score_thres=0.5):
    """Compute mean(IoU) metric
    IoU = intersection / union
    
    For each (mask)threshold in provided range:
     - convert probability mask to boolean mask based on given threshold
     - score the mask 1 if(IoU > score_threshold(0.5))
    Take the mean of the scoress

    https://www.tensorflow.org/api_docs/python/tf/metrics/mean_iou
    """
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        Y_pred_bool = tf.to_int32(Y_pred > t) # boolean mask by threshold
        score, update_op = tf.metrics.mean_iou(Y_true, Y_pred_bool, 2)
        
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            score = tf.identity(score) #!! use identity to transform score to tensor
        prec.append(score) 
    return K.mean(K.stack(prec), axis=0)

# Implementing a parameterized custom loss functions in Keras. 
# 1. writing a method for the coefficient/metric. 
# 2. writing a wrapper function to format things the way Keras needs them to be.

# # import keras.backend as K
# def dice_coef(y_true, y_pred, smooth, thresh):
#     """Dice coefficient

#     Dice = (2*|X & Y|)/ (|X|+ |Y|)
#          =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
#     ref: https://arxiv.org/pdf/1606.04797v1.pdf
#     """
#     y_pred = y_pred > thresh
#     y_pred = K.cast(y_pred, dtype='float32')
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true * y_pred, axis=-1)
# #     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
#     return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


# def dice_loss_(smooth, thresh):
#     """Dice loss factory as Keras loss functions must only take (y_true, y_pred) as parameters."""
#     def dice(y_true, y_pred):
#         return -dice_coef(y_true, y_pred, smooth, thresh)
#     return dice


# def dice_mean_loss(smooth, thresh):
#     """Dice loss factory as Keras loss functions must only take (y_true, y_pred) as parameters."""
#     def dice(y_true, y_pred):
#         return 1 - dice_coef(y_true, y_pred, smooth, thresh)
#     return dice

# Finally, define the loss function to use in model compile
# dice = dice_mean_loss(smooth=1e-5, thresh=0.5) # smooth=1

# Jaccard loss algo
# intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
#     sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
#     jac = (intersection + smooth) / (sum_ - intersection + smooth)
#     return (1 - jac) * smooth

In [ ]:
# smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32') # threshold 0.5
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weight):
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = weight * (logit_y_pred * (1. - y_true) + 
                     K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd
    averaged_mask = K.pool2d(
            y_true, pool_size=(50, 50), strides=(1, 1), padding='same', pool_mode='avg')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight = 5. * K.exp(-5. * K.abs(averaged_mask - 0.5))
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

## Model architecture<a name='J'></a>
<div align="right">[>>TOC](#TOC)</div>

Below functions create the UNet model in a functional and recursive way.

In [ ]:
# Model block - helpers
def conv_block(m, ch_dim, acti, bn, res, do=0):
    """CNN block"""
    n = Conv2D(ch_dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    # http://blog.qure.ai/notes/semantic-segmentation-deep-learning-review - dilation explained
    # https://arxiv.org/pdf/1706.05587.pdf - DeepLab
    n = Conv2D(ch_dim, 3, activation=acti, padding='same', dilation_rate=1)(n)
    o = Conv2D(ch_dim, 3, activation=acti, padding='same', dilation_rate=3)(n) 
#     p = Conv2D(ch_dim, 3, activation=acti, padding='same', dilation_rate=5)(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n, o]) if res else n

def input_feature(f, n, n_features=1):
    """Input block"""
    features = 1
    xx = K.int_shape(n)[1]
    f_repeat = RepeatVector(xx*xx)(f)
    f_conv = Reshape((xx, xx, n_features))(f_repeat)
    n = Concatenate(axis=-1, name=f'feat_{2}')([n, f_conv])
    n = BatchNormalization()(n)            
    return n

def input_csum(f, n, n_features=1):
    """Input block"""
    features = 1
    xx = K.int_shape(n)[1]
    n = Concatenate(axis=-1, name=f'csum_{2}')([n, f])
    n = BatchNormalization()(n)            
    return n

In [ ]:
def level_block(m, ch_dim, depth, inc_rate, acti, do, bn, mp, up, res, inp_feat, inp_csum):
    """Recursive CNN builder"""
    if depth > 0:
        n = conv_block(m, ch_dim, acti, bn, res) # add/remove drop-out: ,do
        m = MaxPooling2D()(n) if mp else Conv2D(ch_dim, 3, strides=2, padding='same')(n)
        # Input csum at 3rd layer (32x32)
        if (inp_csum is not None) and (depth==DEPTH):
            m = Concatenate()([m, input_csum(inp_csum, m)])
        # Input depth
        if (inp_feat is not None) and (depth==1):
            m = Concatenate()([m, input_feature(inp_feat, m)])
        m = level_block(m, int(inc_rate*ch_dim), depth-1, inc_rate, acti, do, bn, mp, up, res, inp_feat, inp_csum)
        
        # Unwind recursive stack calls - creating the upscaling part of the model
        if up:
            # Repeat the rows and columns of the data by 2 and 2 respectively - untrainable like reverse pooling
            m = UpSampling2D()(m)
            m = Conv2D(ch_dim, 2, activation=acti, padding='same')(m)
        else:
            # Transposed convolutions are going in the opposite direction of a normal convolution - trainable
            m = Conv2DTranspose(ch_dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, ch_dim, acti, bn, res)
    else:
        # Depth == 0 - deepest conv_block
        m = conv_block(m, ch_dim, acti, bn, res, do) # add/remove drop-out: ,do
    return m

In [ ]:
def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
        dropout=0.5, batchnorm=False, maxpool=True, upconv=False, residual=False):
    """Returns model"""
    inputs = Input(shape=img_shape, name='img')
    inp_csum = None # Input(shape=(64, 64,1), name='csum') ###
    inp_feat = None # Input(shape=(1,), name='feat') # or None
    outputs = level_block(inputs, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual, inp_feat, inp_csum)
    outputs = Conv2D(out_ch, 1, activation='sigmoid')(outputs)
#     return Model(inputs=[inputs, inp_csum, inp_feat], outputs=outputs)
    return Model(inputs=[inputs], outputs=outputs)

## Build model<a name='K'></a>
<div align="right">[>>TOC](#TOC)</div>

Interesting reads:  
[orginal paper](https://arxiv.org/pdf/1505.04597.pdf)  
[Upsampling2D vs. Convolution2DTranspose](#https://distill.pub/2016/deconv-checkerboard/)  
[Optimizers](http://ruder.io/optimizing-gradient-descent/)  

In [ ]:
IMG_CH = 1                  # X_train.shape[-1]  # layers of image
CONV_CH = 16                # number of channels to start/end UNet with
DEPTH = 4                   # number of CONV blocks to max model depth - outstride 8: 128/8 = 16(depth=4), outstride 16: 128/16 = 8(depth=3), depth 3 doesnot work well
D_OUT = 0.2                 # small random effect can be effective against overfitting. 
                            # Drop-out layers at the end of the contracting path perform further implicit data augmentation.
BN = True
UP_CONV = False             # UPSampling avoids coarse checkerboard effect in images
RES = True                  # residuals skip layers for deep networks (gradient evaporation)

model = UNet((TGT_SIZE, TGT_SIZE, IMG_CH), 
             start_ch=CONV_CH, 
             depth=DEPTH, 
             dropout=D_OUT,
             batchnorm=BN, 
             upconv=UP_CONV,
             residual=RES)

### Compile and visualize model

In [ ]:
LR = 0.001
# Define optimizer
# Clip gradients to norm 1., 
optimizer = {'adam': Adam(lr=LR, beta_1=0.9, beta_2=0.999, clipnorm=8.),
             'sgd': SGD(lr=LR, decay=LR/100, momentum=0.99, nesterov=True, clipnorm=5.),
             'rmsprop': RMSprop(lr=LR, rho=0.9, epsilon=None, decay=0.0),
             'adadelta': Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
             'nadam': Nadam(lr=LR, beta_1=0.9, beta_2=0.999)} # , epsilon=None, schedule_decay=0.004
opt = 'nadam'

# Define loss
# TODO Dice needs debugging
loss = ["binary_crossentropy", "kullback_leibler_divergence", dice_loss, bce_dice_loss, bce_logdice_loss]

# Compile model
model.compile(loss=loss[3], optimizer=optimizer[opt], metrics=["accuracy", mean_iou]) #accuracy

model_name = f'TGS_UNet_{opt}_CH-{IMG_CH}-{CONV_CH}_D-{DEPTH}_DO-{D_OUT>0}_BN-{BN}_UP-{UP_CONV}_RES-{RES}.h5'
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(patience=15, verbose=1), # bias-variance tradeof
    ReduceLROnPlateau(patience=4, min_lr=0.00001, verbose=1),
    ModelCheckpoint(model_name, monitor='val_mean_iou', mode='max', save_best_only=True, verbose=1)] # continue/save based on monitor metric

## Train validation split<a name='H'></a>
<div align="right">[>>TOC](#TOC)</div>

Validation set will be used for saving model checkpoints and early stopping. 
Tradeoff:
 - larger validation set for more validation accuracy and saving better generalizing model, which means less training samples and thus less generalizing model.

Using images with depth layer.

In [ ]:
# Split train val indices
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(train_indices, test_size=0.15, random_state=1, stratify=Y_cov_class)

## Train model<a name='L'></a>
<div align="right">[>>TOC](#TOC)</div>

Model can be trained succesively

In [ ]:
# First fit
BATCH_SIZE = 24
EPOCHS = 5
history1 = model.fit_generator(data_gen(batch=BATCH_SIZE, indices=train_ids.tolist(), dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train),
                              epochs=EPOCHS,
                              steps_per_epoch=len(train_ids) //BATCH_SIZE,
                              validation_data=val_gen(batch=BATCH_SIZE, indices=val_ids.tolist(), dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train),
                              validation_steps=len(val_ids) //BATCH_SIZE,
                              verbose=1, 
                              callbacks=callbacks,
                              workers=-1)    

# Second fit - can change the augmentation, batch size
BATCH_SIZE = 16
EPOCHS_2 = 0 # total no of epochs incl. first training session
if EPOCHS_2 > EPOCHS:
    history2 = model.fit_generator(data_gen(batch=BATCH_SIZE, indices=train_ids.tolist(), dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train),
                                  initial_epoch=EPOCHS,  # resume from earlier training
                                  epochs=EPOCHS_2,
                                  steps_per_epoch=len(train_ids) //BATCH_SIZE,
                                  validation_data=val_gen(batch=BATCH_SIZE, indices=val_ids.tolist(), dim=TGT_SIZE, x_path=imgs_train, y_path=masks_train),
                                  validation_steps=len(val_ids) //BATCH_SIZE,
                                  verbose=1, 
                                  callbacks=callbacks,
                                  workers=-1)   

## Learning curves<a name='M'></a>
<div align="right">[>>TOC](#TOC)</div>

1. Check the curves for errors  
2. Check the curves for roughness and convergence to tune the hyperparameters:  
 - learning rate and decay
 - batch size
 - model architecture
 - epochs and earlystopping

In [ ]:
# Learning curves
for history in [history1,]: # history2]:
    fig, (ax_loss, ax_acc, ax_iou) = plt.subplots(1, 3, figsize=(20,8))
    _ = ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
    _ = ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
    _ = ax_loss.legend()
    _ = ax_loss.set_title('Loss')

    _ = ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
    _ = ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")
    _ = ax_acc.legend()
    _ = ax_acc.set_title('Accuracy')

    _ = ax_iou.plot(history.epoch, history.history["mean_iou"], label="Train IoU")
    _ = ax_iou.plot(history.epoch, history.history["val_mean_iou"], label="Validation IoU")
    _ = ax_iou.legend()
    _ = ax_iou.set_title('IoU')

## Check performance on validation set <a name='N'></a>
<div align="right">[>>TOC](#TOC)</div>

In [ ]:
# Load best model
model = load_model(model_name, custom_objects={'mean_iou': mean_iou})
print('model loading done.')

In [ ]:
# Load validation set
X_val = np.array([np.array(upsample(np.array(load_img(f'{imgs_train}/{name}.png', grayscale=True)), TGT_SIZE) / 255)[...,np.newaxis] for name in val_ids])
Y_val = np.array([np.array(upsample(np.array(load_img(f'{masks_train}/{name}.png', grayscale=True)), TGT_SIZE) / 255)[...,np.newaxis] for name in val_ids])

In [ ]:
if DEV:
    print(X_val.shape, Y_val.shape)

In [ ]:
# Evaluate best model on validation set
# DEV = True
if DEV:
    val_scores = model.evaluate(X_val, Y_val, verbose=1)
    print(f'model validation scores: \nval_loss: {val_scores[0]}, \nval_acc: {val_scores[1]}, \nval_miou: {val_scores[2]}')


In [ ]:
# Predict on validation set
preds_valid = model.predict(X_val, verbose=1).reshape(-1, TGT_SIZE, TGT_SIZE)
preds_valid = preds_valid.reshape(-1, TGT_SIZE, TGT_SIZE, 1)

In [ ]:
if DEV:
    print(preds_valid.shape) #, Y_valid.shape)

### Visualize validation(probability masks) layed over GT masks

Grount truth: green mask
Validation: pink mask

- Green are false positives (FP)
- Pink are false negatives (FN)
- Brown are true positives (TP)
- Grey are true negatives (TN)

In [ ]:
# Threshold = .6
if DEV:
    N = np.arange(100)
    plot_imgs_masks(X_val[N], Y_val[N], preds_valid=preds_valid[N]>.6, title='GT(green) vs.Validation mask')

## Post-processing: blur and threshold<a name='Q'></a>
<div align="right">[>>TOC](#TOC)</div>

In [ ]:
# Postprocessing helper functions
def adaptive_gaus_thres(img, n=5, m=0):
    """Adaptive Gaussian thresholding
    cv2.threshold requires a 1 layer uint8 image"""
    img = np.array(img, dtype=np.uint8)
    return cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, n, m)


def adaptive_mean_thres(img, n=5, m=0):
    """Adaptive Mean thresholding"""
    img = np.array(img, dtype=np.uint8)
    return cv2.adaptiveThreshold(img, 1, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, n, m)


def otsu_thres(img, n=0, m=255):
    """Otsu thresholding"""
    img = np.array(img, dtype=np.uint8)
    return cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


def gaus_thres(img, n=0, m=255):
    """Gaussian thresholding"""
    img = np.array(img, dtype=np.uint8)
    return cv2.threshold(img, n, m, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)[1]


def plot_mask_thres(fn, img, n, m, cmap='seismic', title=''):
    """Plot thresholding images"""
    plt.imshow(fn(img, n, m), cmap=cmap); plt.title(title)
    plt.show();

In [ ]:
if DEV:
    # Compare masks with different blur
    for i in random.sample(range(Y_val.shape[0]), 2):
        y_pred = preds_valid[...,0][i]
        y = Y_val[...,0][i]
        blurr = 7 # restricted in medianBlur - 5 is max
        thres = .7
        fig, ax = plt.subplots(1,4, figsize=(24,6))

        _ = ax[0].imshow(y>thres, cmap="seismic"); ax[0].set_title('Ground Truth mask')

        _ = ax[1].imshow(y_pred>thres, cmap="seismic"); ax[1].set_title('Prediction mask')
        _ = ax[1].imshow(y>thres, alpha=.3, cmap="Blues")

        img_blur = cv2.medianBlur(y_pred, 5)
        _ = ax[2].imshow(img_blur>thres, cmap="seismic"); ax[2].set_title('Median blur')
        _ = ax[2].imshow(y>thres, alpha=.3, cmap="Blues")

        img_blur = cv2.blur(y_pred, (blurr, blurr))
        _ = ax[3].imshow(img_blur>thres, cmap="seismic"); plt.title('Gaussian blur')
        _ = ax[3].imshow(y>thres, alpha=.3, cmap="Blues")
        _ = plt.show();

In [ ]:
if DEV:
    # Combinations of blur and threshold
    sample = random.sample(range(Y_val.shape[0]), 2)
    for i in sample:
        thres = .7
        blur_kernel = (7, 7)
        Y_pred_gaus = np.array([cv2.blur(y, blur_kernel) for y in preds_valid]).reshape(preds_valid.shape)
        
        y = Y_val[...,0][i]
        y_pred = preds_valid[...,0][i]
        y_gaus = Y_pred_gaus[...,0][i]

        fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7) = plt.subplots(1,7, figsize=(25,5), sharey=True)
        # GT mask
        _ = ax1.imshow(y, cmap="seismic"); 
        _ = ax1.set_title('Ground truth mask')

        # Probability mask
        _ = ax2.imshow(y_pred, cmap="seismic"); 
        _ = ax2.set_title('Probability mask')
        _ = ax2.imshow(y, alpha=.3, cmap="Blues")

        # Prediction mask
        img = y_pred > thres
        _ = ax3.imshow(img, cmap="seismic"); 
        _ = ax3.set_title('Prediction mask')
        _ = ax3.imshow(y, alpha=.3, cmap="Blues")

        # Opencv thresh prediction mask
        thres, img = cv2.threshold(y_pred, thres, 1, cv2.THRESH_BINARY)
        _ = ax4.imshow(img, cmap="seismic"); 
        _ = ax4.set_title('Opencv threshold')
        _ = ax4.imshow(y, alpha=.3, cmap="Blues")

        # Blur>thres prediction mask
        thres, img = cv2.threshold(y_gaus, thres, 1, cv2.THRESH_BINARY)
        _ = ax5.imshow(img, cmap="seismic"); 
        _ = ax5.set_title('Blur>thres')
        _ = ax5.imshow(y, alpha=.3, cmap="Blues")

        # Thres>blur>thres prediction mask
        thres, img = cv2.threshold(y_pred, thres, 1, cv2.THRESH_BINARY)
        img = cv2.blur(img, blur_kernel) > thres
        _ = ax6.imshow(img, cmap="seismic"); 
        _ = ax6.set_title('Thres>blur>thres')
        _ = ax6.imshow(y, alpha=.3, cmap="Blues")

        # Blur>thres>blur>thres prediction mask
        thres, img = cv2.threshold(y_gaus, thres, 1, cv2.THRESH_BINARY)
        img = cv2.blur(img, blur_kernel) > thres
        _ = ax7.imshow(img, cmap="seismic"); 
        _ = ax7.set_title('Blur>thres>blur>thres')
        _ = ax7.imshow(y, alpha=.3, cmap="Blues")
        _ = plt.show();

In [ ]:
if DEV:
    # Experiment with special thresholding algo's
    for i in random.sample(range(Y_val.shape[0]), 5):
        thres = .7
        blur_kernel, blurr = (3, 3), 3
        Y_pred_gaus = np.array([cv2.blur(y, blur_kernel) for y in preds_valid]).reshape(preds_valid.shape)
        
        y = Y_val[...,0][i]
        y_pred = preds_valid[...,0][i]
        y_pred = (255 * y_pred).astype(np.uint8)
        y_gaus = Y_pred_gaus[...,0][i]

        fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1,6, figsize=(25,5), sharey=True)
        # GT mask
        _ = ax1.imshow(y, cmap="seismic"); 
        _ = ax1.set_title('Ground truth mask')
        
        # Prediction mask
        Y_ = y_pred > thres
        _ = ax2.imshow(Y_, cmap="seismic"); 
        _ = ax2.set_title('Prediction mask')
        _ = ax2.imshow(y, alpha=.3, cmap="Blues")

        # Otsu prediction mask
        _, Y_otsu = cv2.threshold(y_pred, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        _ = ax3.imshow(Y_otsu, cmap="seismic"); 
        _ = ax3.set_title('Otsu mask')
        _ = ax3.imshow(y, alpha=.3, cmap="Blues")
        
        # Adaptive threshold
        _, Y_gaus = cv2.threshold(y_pred, 60, 240, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)
        _ = ax4.imshow(cv2.bitwise_not(Y_gaus), cmap="seismic"); 
        _ = ax4.set_title('Gaussian threshold')  # reverse image: cv2.bitwise_not()
        _ = ax4.imshow(y, alpha=.3, cmap="Blues")
        
        # Compbined Otsu and Adaptive
        Y_mean = (Y_otsu+Y_gaus)/2
        _ = ax5.imshow(cv2.bitwise_not(Y_mean), cmap="seismic"); 
        _ = ax5.set_title('IoU Otsu and Gaussian') 
        _ = ax5.imshow(y, alpha=.3, cmap="Blues")
        
        #
        img_thresh_Gaussian = cv2.adaptiveThreshold(y_pred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, 1)
        _ = ax6.imshow(img_thresh_Gaussian, cmap="seismic"); 
        _ = ax6.set_title('Adaptive Gaussian')
        _ = ax6.imshow(y, alpha=.3, cmap="Blues")

In [ ]:
# Helper functions for Otsu masking
def pred_otsu(y, thresh, blurr=3):
    """Preprocess mask for otsu threshold
    Returns:
    binary mask"""
    blur_kernel = (blurr, blurr)
    try:
        im = cv2.blur(y, blur_kernel) > thresh
        im = np.array(im * 255, dtype=np.uint8)
        _, Y_otsu = cv2.threshold(im, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    except:
        Y_otsu = np.zeros_like(y)
    return Y_otsu//255

def preds_otsu(Y, threshold, blurr):
    """Preprocess masks for otsu threshold"""
    Y_otsu = [pred_otsu(y, threshold, blurr) for y in Y[...,0]]
    return np.array(Y_otsu, dtype=np.int8).reshape(Y.shape)

# Sanity check function
if DEV:
    for i in [6, 13, 17, 27]:
        thres = .7
        blurr = 3
        _ = plt.imshow(preds_otsu(preds_valid[:28], threshold=thres, blurr=blurr)[i,...,0], cmap="seismic")
        _ = plt.imshow(Y_val[...,0][i], alpha=.3, cmap="Blues")
        _ = plt.title('Otsu'); 
        _ = plt.show();

## Scoring<a name='O'></a>
<div align="right">[>>TOC](#TOC)</div>

Score the model and do a threshold optimization by the best IoU

In [ ]:
def IoU(Y_true, Y_pred):
    """IoU of unary/boolean mask using confusion matrix"""
    CM, _, _ = np.histogram2d(Y_true.ravel(), Y_pred.ravel(), bins=(2,2))
    TN, FN, FP, TP = np.split(CM.ravel(), 4)
    if TP==0:
        return 0
    else:
        iou = TP / (FN + FP + TP)
    return np.mean(np.arange(0.5, 1.0, 0.05) < iou)

def miou(Y_trues, Y_preds):
    """Mean intersection over union of all masks"""
    return np.mean([IoU(Y_trues[i], Y_preds[i]) for i in range(Y_trues.shape[0])])

def best_miou(mious):
    """Returns tuple of (best IoU, corresponding threshold)"""
    best_idx = np.argmax(mious)
    return mious[best_idx], thresholds[best_idx]

### Best threshold for best IoU score (submission)

Checking which threshold delivers the best IoU score, so this threshold will be used for Test set prediction.

### Compare blur effect on IoU

In [ ]:
if DEV:
    # Explore best params giving best IoU: blur kernel size, threshold
    blur_range = [1, 3, 5]
    mious_otsu = {}
    for i, blurr in enumerate(blur_range):
        print(f'computing best IoU @blur: {blurr}')
        thresholds = np.linspace(0.1, 0.9, 80)
        mious_otsu = {**mious_otsu, **{blurr: {threshold: miou(Y_val, preds_otsu(preds_valid, threshold, blurr=blurr))
                     for threshold in tqdm_notebook(thresholds)}}}

In [ ]:
# 
if DEV:
    # Visualize IoU by blur and as function of thresholds
    _ = plt.figure(figsize=(20,10))
    _ = plt.xlabel("Threshold")
    _ = plt.ylabel("IoU")
    _ = plt.title(f"Mean-IoU")
    
    for i in blur_range:
        best_idx = np.argmax(list(mious_otsu[i].values()))
        threshold_best_ = list(mious_otsu[i])[best_idx]
        miou_best_ = list(mious_otsu[i].values())[best_idx]
        _ = plt.plot(list(mious_otsu[i]), mious_otsu[i].values(), label=f'Blur: {i}')
        _ = plt.plot(threshold_best_, miou_best_, "ok", label=f"Best threshold: {threshold_best_.round(3)}")
        _ = plt.legend()

In [ ]:
# IoU scores for best params
# Blur 3 gives same reuslts as more blur, this is the savest bet
blurr = 3 
thresholds = np.linspace(0.1, 0.9, 80)
mious_otsu = np.array([miou(Y_val, preds_otsu(preds_valid, threshold, blurr=blurr))
                 for threshold in tqdm_notebook(thresholds)])

# Get best scores by threshold
miou_best_otsu, threshold_best_otsu = best_miou(mious_otsu)

if DEV:
    mious_no = np.array([miou(Y_val, np.int8(preds_valid > threshold)) 
                     for threshold in tqdm_notebook(thresholds)])
    miou_best, threshold_best = best_miou(mious_no)

In [ ]:
if True: #DEV:
    # Visualize best IoU as function of thresholds
    plt.figure(figsize=(20,10))
    _ = plt.plot(thresholds, mious_otsu, label=f'Otsu threshold with blur {blurr}')
    _ = plt.plot(thresholds, mious_no, label='No blur')
    _ = plt.plot(threshold_best, miou_best, "xr", label=f"Best threshold: {threshold_best.round(3)}")
    _ = plt.plot(threshold_best_otsu, miou_best_otsu, "xk", label=f"Best threshold otsu: {threshold_best_otsu.round(3)}")
    _ = plt.xlabel("Threshold")
    _ = plt.ylabel("IoU")
    _ = plt.title(f"Mean-IoU: {miou_best.round(3)}(default) - {miou_best_otsu.round(3)}(otsu)")
    _ = plt.legend()

### Visualize binary masks based on best threshold

- Green are false positives (FP)
- Pink are false negatives (FN)
- Brown are true positives (TP)
- Grey are true negatives (TN)


In [ ]:
# Default threshold
if DEV:
    N = np.arange(60)
    plot_imgs_masks(X_val[N], Y_val[N], preds_valid=preds_valid[N], thres=threshold_best, title='GT vs. Validation masks - best threshold')

In [ ]:
# Otsu threshold
if DEV:
    N = np.arange(60)
    blurr = 5
    thres = .5
    plot_imgs_masks(X_val[N], Y_val[N], preds_valid=preds_otsu(preds_valid, threshold_best_otsu, blurr=blurr)[N], 
                    thres=threshold_best_otsu, title='GT vs. Validation masks - best Otsu threshold')

In [ ]:
# Visualize effect of using Otsu threshold
if DEV:
    N = np.arange(60)
    blurr = 5

    Y_pred_non = np.array([y for y in preds_valid[N]]).reshape(preds_valid[N].shape)
    Y_pred_otsu = preds_otsu(preds_valid, threshold_best_otsu, blurr=blurr)[N]

    plot_imgs_masks((np.zeros_like(Y_pred_non)), Y_pred_non, preds_valid=Y_pred_otsu, thres=threshold_best_otsu, title='Overlay validation and Otso')
    diff = abs(Y_pred_non - Y_pred_otsu) - np.ones_like(Y_pred_non)
    plot_imgs_masks((np.zeros_like(Y_pred_non)), diff, preds_valid=diff, title='Difference between validation and Otsu')



In [ ]:
# Free up memory: need ~7gb for test set
gc.collect()
print(f'notebook memory used: {mem_used()}mb')

## Predict test set<a name='P'></a>
<div align="right">[>>TOC](#TOC)</div>

### Sanity check if all indices and images match

In [ ]:
if DEV:
    test_ids = next(os.walk(f"{path_test}/images"))[2]
    print(set(test_ids) ^ set(test_df.index+'.png'))
    assert len(set(test_ids) ^ set(test_df.index+'.png')) == 0

> ### Convert test set images and depths to layers

 - Upsample images to arrays
 - Reshape for modeling
 - Create depth layer

In [ ]:
# Test set has 17570/18000 = 97,6% non-empty images
X_test = np.array([upsample(np.array(load_img(f'{imgs_test}/{name}.png', grayscale=True)), TGT_SIZE) / 255  # image vector
              for name in tqdm_notebook(test_indices)])[...,np.newaxis]


In [ ]:
# Predict X_test - keep around for multi model input
# X_test_dict = {'img': X_test, 
#                'csum': X_test_csum,
#                'feat': X_test_norm_depth}

Y_test = model.predict(X_test) # probability
Y_test_otsu = preds_otsu(Y_test, threshold_best_otsu, 3) # is already unary/boolean!
print('prediction done.')

### Visually check of predictions

In [ ]:
# Visual check of predictions without/with applying Otsu threshold
if DEV:
    N = np.arange(60)
    plot_imgs_masks(X_test[N], np.zeros_like(X_test[N]), preds_valid=Y_test[N], thres=threshold_best, title='Predicted masks - best threshold')
    plot_imgs_masks(X_test[N], np.zeros_like(X_test[N]), preds_valid=Y_test_otsu[N], thres=threshold_best_otsu, title='Predicted masks - best Otsu threshold')

## Submission  <a name='R'></a> 
<div align="right">[>>TOC](#TOC)</div>
Submission is in csv form:
 - `id`: index (equals filename)
 - `rle_mask`: run-length format (down-then-right): `masked_pixel_start` `<space>` `length_of_masked_pixels` ...

In [ ]:
def RLenc(img, order='F'):
    """Convert binary mask image to run-length array or string.
    
    Args:
    img: image in shape [n, m]
    order: is down-then-right, i.e. Fortran(F)
    string: return in string or array

    Return:
    run-length as a string: <start[1s] length[1s] ... ...>
    """
    bytez = img.reshape(img.shape[0]*img.shape[1], order=order)
    bytez = np.concatenate([[0], bytez, [0]])
    runs = np.where(bytez[1:] != bytez[:-1])[0] + 1 # pos start at 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Use for sanity check the encode function
def RLdec(rl_string, shape=(101, 101), order='F'):
    """Convert run-length string to binary mask image.
    
    Args:
    rl_string: 
    shape: target shape of array
    order: decode order is down-then-right, i.e. Fortran(F)

    Return:
    binary mask image as array
    """
    s = rl_string.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order=order)

In [ ]:
def downsample(img, dim=101):
    """Downsize an image"""
    return cv2.resize(img, (dim, dim), interpolation=cv2.INTER_NEAREST)

def downsample_(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

In [ ]:
# Encode predictions on Otsu validation threshold - upsample doesnot work (??)
pred_dict = {idx: RLenc(downsample(Y_test_otsu[i], dim=101))
             for i, idx in enumerate(tqdm_notebook(test_indices))}

In [ ]:
if DEV:
    pd.DataFrame.from_dict(pred_dict, orient='index').sample(10)
    
pd.DataFrame.from_dict(pred_dict, orient='index').sample(10)

### Save submission

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict, orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')
print('submission saved!')

if DEV:
    sub.sample(10)

### Thanks for upvoting and sharing your thoughts!